In [1]:
import pandas as pd
df_train = pd.read_csv('/Users/jackcapt/Documents/BIT STUFF/Python script/Weibo Data/weibo_train_data.txt', delimiter = "\t", encoding="utf-8")
df = pd.DataFrame(df_train)
df = df.rename(columns={"d38e9bed5d98110dc2489d0d1cac3c2a": "uid", "7d45833d9865727a88b960b0603c19f6": "mid", "2015-02-23 17:41:29": "time","0":"forward_count","0.1":"comment_count","0.2":"like_count","丽江旅游(sz002033)#股票##炒股##财经##理财##投资#推荐包赢股，盈利对半分成，不算本金，群：46251412":"content"}, errors="raise")
df.head()

,uid,mid,time,forward_count,comment_count,like_count,content
0,fa13974743d3fe6ff40d21b872325e9e,8169f1d45051e08ef213bf1106b1225d,2015-02-14 12:49:58,0,0,0,#丁辰灵的红包#挣钱是一种能力，抢红包拼的是技术。我抢到了丁辰灵 和@阚洪岩 一起发出的现金...
1,da534fe87e7a52777bee5c30573ed5fd,68cd0258c31c2c525f94febea2d9523b,2015-03-31 13:58:06,0,0,0,淘宝网这些傻逼。。。气的劳资有火没地儿发~尼玛，你们都瞎了
2,e06a22b7e065e559a1f0bf7841a85c51,00b9f86b4915aedb7db943c54fd19d59,2015-06-11 20:39:57,0,4,3,看点不能说的，你们都懂[笑cry]
3,f9828598f9664d4e347ef2048ce17734,c7f6f66044c0c5a3330e2c5371be6824,2015-03-10 18:02:38,0,0,0,111多张
4,d80f3d3c5c1d658e82b837a4dd1af849,bfc0819b83ec59ce767287077f2b3507,2015-02-13 01:09:41,0,0,0,有生之年！我最喜欢的up主跟我的三体勾搭到一起了！幸福感爆棚！ @黑桐谷歌 http://...


In [2]:
context = df[['content']]
context.head()

,content
0,#丁辰灵的红包#挣钱是一种能力，抢红包拼的是技术。我抢到了丁辰灵 和@阚洪岩 一起发出的现金...
1,淘宝网这些傻逼。。。气的劳资有火没地儿发~尼玛，你们都瞎了
2,看点不能说的，你们都懂[笑cry]
3,111多张
4,有生之年！我最喜欢的up主跟我的三体勾搭到一起了！幸福感爆棚！ @黑桐谷歌 http://...


In [3]:
#Usable
import re
def clean_data(context):
#replace URL of a text
    #Remove links
    context['content'] = context['content'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '')
    #Remove Emoji
    context['content'] = context['content'].str.replace('\[.{0,12}\]', '')
    #Remove Hashtags
    tags = re.findall("#(.{0,30})#",str(context))
    context['content'] = context['content'].str.replace('#(.{0,30})#', '')
    #Extract&Remove EnglishCharacters
    english = re.findall("[a-z]+",str(context))
    context['content'] = context['content'].str.replace('[a-z]+', '')
    #Extract and remove Mentions
    at = re.findall("@([^@]{0,30})\s",str(context))
    context['content'] = context['content'].str.replace('@([^@]{0,30})\s', '')
    at+= re.findall("@([^@]{0,30})）",str(context))
    context['content'] = context['content'].str.replace('@([^@]{0,30})）', '')
    #Remove Spaces
    context['content'] = context['content'].str.replace('\s', '')
    #Remove Digits
    context['content'] = context['content'].str.replace('\d', '')
    #Remove……
    context['content'] = context['content'].str.replace('\.*', '')
    #Remove Punctuation
    context['content'] = context['content'].str.replace('[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+', '')
    context['content'] = context['content'].str.replace('[【】╮╯▽╰╭★→「」]+', '')
    context['content'] = context['content'].str.replace('！，❤。～《》：（）【】「」？”“；：、', '')

clean_data(context)
print(context['content']);

<ipython-input-3-fab2589a3a9d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  context['content'] = context['content'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '')
<ipython-input-3-fab2589a3a9d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  context['content'] = context['content'].str.replace('\[.{0,12}\]', '')
<ipython-input-3-fab2589a3a9d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

0          挣钱是一种能力抢红包拼的是技术我抢到了丁辰灵和一起发出的现金红包幸福感爆棚情人节一起来和粉丝...
1                                   淘宝网这些傻逼气的劳资有火没地儿发尼玛你们都瞎了
2                                                 看点不能说的你们都懂
3                                                         多张
4                                 有生之年我最喜欢的主跟我的三体勾搭到一起了幸福感爆棚
                                 ...                        
1225082                             PHP设计模式：命令C模式--简明现代魔法分享自
1225083                                        金鱼的记忆只有七秒股民也是
1225084                                                     
1225085     小年抢个好彩头我在五岳散人的红包中抽到了提供的“快的打车元红包”羊年未到好运先来你也快来试试手气
1225086    电子土拨鼠|当虚拟撞上现实技术宅逆袭浪漫求婚今年月里的一次求婚相信你肯定会记得：汪峰在章子怡...
Name: content, Length: 1225087, dtype: object


<ipython-input-3-fab2589a3a9d>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  context['content'] = context['content'].str.replace('！，❤。～《》：（）【】「」？”“；：、', '')


In [4]:
context.head()

,content
0,挣钱是一种能力抢红包拼的是技术我抢到了丁辰灵和一起发出的现金红包幸福感爆棚情人节一起来和粉丝...
1,淘宝网这些傻逼气的劳资有火没地儿发尼玛你们都瞎了
2,看点不能说的你们都懂
3,多张
4,有生之年我最喜欢的主跟我的三体勾搭到一起了幸福感爆棚


In [5]:
cleaned_data = pd.concat([df,context], axis=1, sort=False)
cleaned_data.head()

,uid,mid,time,forward_count,comment_count,like_count,content,content
0,fa13974743d3fe6ff40d21b872325e9e,8169f1d45051e08ef213bf1106b1225d,2015-02-14 12:49:58,0,0,0,#丁辰灵的红包#挣钱是一种能力，抢红包拼的是技术。我抢到了丁辰灵 和@阚洪岩 一起发出的现金...,挣钱是一种能力抢红包拼的是技术我抢到了丁辰灵和一起发出的现金红包幸福感爆棚情人节一起来和粉丝...
1,da534fe87e7a52777bee5c30573ed5fd,68cd0258c31c2c525f94febea2d9523b,2015-03-31 13:58:06,0,0,0,淘宝网这些傻逼。。。气的劳资有火没地儿发~尼玛，你们都瞎了,淘宝网这些傻逼气的劳资有火没地儿发尼玛你们都瞎了
2,e06a22b7e065e559a1f0bf7841a85c51,00b9f86b4915aedb7db943c54fd19d59,2015-06-11 20:39:57,0,4,3,看点不能说的，你们都懂[笑cry],看点不能说的你们都懂
3,f9828598f9664d4e347ef2048ce17734,c7f6f66044c0c5a3330e2c5371be6824,2015-03-10 18:02:38,0,0,0,111多张,多张
4,d80f3d3c5c1d658e82b837a4dd1af849,bfc0819b83ec59ce767287077f2b3507,2015-02-13 01:09:41,0,0,0,有生之年！我最喜欢的up主跟我的三体勾搭到一起了！幸福感爆棚！ @黑桐谷歌 http://...,有生之年我最喜欢的主跟我的三体勾搭到一起了幸福感爆棚


In [6]:
#cleaned_data.to_csv('/Users/jackcapt/Documents/BIT STUFF/Python script/Weibo Data/df_tranClean.csv', index=False)